In [1]:
# Find bets placed
# Check order
# Create True False
# Check win yes / no

In [106]:
# import libraries
from datetime import datetime
from pytz import timezone
import time
import requests
import pandas as pd
from datetime import date
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()

# chrome_options.add_argument("--headless")
chrome_options.headless = True # also works
PATH = 'C:\\Users\\devna\\Ecotec\\chromedriver.exe'
driver = webdriver.Chrome(PATH, options=chrome_options)

url = 'https://www.oddsportal.com/basketball/usa/nba/results/'

odds_df = ''
odds_list = []

In [107]:
# Change timezone to Eastern USA
driver.get('https://www.oddsportal.com/basketball/usa/nba/')
time.sleep(3)
timezone_option = driver.find_elements_by_class_name('user-header-fakeselect')
timezone_option[1].click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[7]/div/a[40]').click()

driver.get(url)
time.sleep(3)

table = driver.find_element_by_tag_name('tbody')

rows = table.find_elements_by_tag_name('tr')

for row in rows:
    try:
        date = row.find_element_by_css_selector('th.first2.tl').text
    except:
        cols = row.find_elements_by_tag_name('td')

        if len(cols) > 5:
            teams = cols[1].text
            score = cols[2].text
            home_odds = cols[3].text
            vis_odds = cols[4].text
            odds_dict = {
                'Date': date,
                'Teams': teams,
                'Score': score,
                'Home Odds': home_odds,
                'Vis Odds': vis_odds
            }
            odds_list.append(odds_dict)
            
odds_df = pd.DataFrame(odds_list)

In [108]:
# Keep only todays
odds_df = odds_df[odds_df['Date'].str.contains('Yesterday')]
odds_df.drop_duplicates(inplace=True)

# Manipulate the data

# Split Teams into Home and Away
odds_df[['Home','Visitor']] = odds_df['Teams'].str.split("-",expand=True,)
odds_df['Home'] = odds_df['Home'].str.strip().replace('\n', '')
odds_df['Visitor'] = odds_df['Visitor'].str.strip()

# Split Score into home and away
odds_df['Score'] = odds_df['Score'].str.replace('OT', '')
odds_df[['Home PTS', 'Vis PTS']] = odds_df.Score.str.split(':', expand=True)
odds_df

# Drop Teams  and Score column column
odds_df = odds_df.drop(['Teams', 'Score'], axis=1)

odds_df = odds_df.dropna(subset=['Visitor', 'Vis PTS'])
today = datetime.now(timezone('EST')).strftime('%Y-%m-%d')
odds_df['Date'] = today

odds_df['Home Win'] = odds_df['Home PTS'] > odds_df['Vis PTS']
odds_df

In [109]:
# Get order from future predictions
# merge
# print out new xslx

,Date,Home Odds,Vis Odds,Home,Visitor,Home PTS,Vis PTS,Home Win
1,2021-05-08,1.37,3.27,Phoenix Suns,New York Knicks,128,105,True
2,2021-05-08,1.31,3.72,Portland Trail Blazers,Los Angeles Lakers,106,101,True
3,2021-05-08,2.61,1.54,Sacramento Kings,San Antonio Spurs,104,113,False
4,2021-05-08,1.57,2.52,Utah Jazz,Denver Nuggets,127,120,True
5,2021-05-08,1.17,5.52,Dallas Mavericks,Cleveland Cavaliers,110,90,False
6,2021-05-08,1.31,3.63,Charlotte Hornets,Orlando Magic,122,112,True
7,2021-05-08,1.29,3.80,Miami Heat,Minnesota Timberwolves,121,112,True
8,2021-05-08,1.04,13.28,Milwaukee Bucks,Houston Rockets,141,133,True
9,2021-05-08,2.51,1.57,Chicago Bulls,Boston Celtics,121,99,False
10,2021-05-08,1.13,6.63,Philadelphia 76ers,New Orleans Pelicans,109,107,True


C:\Users\devna\anaconda3\envs\AI\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)
C:\Users\devna\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  This is separate from the ipykernel package so we can avoid doing imports until
